In [2]:
# from transformers import DistilBertTokenizer, DistilBertForMaskedLM, DataCollatorForLanguageModeling, Trainer, EarlyStoppingCallback, TrainingArguments
from transformers import DistilBertTokenizer, DistilBertForMaskedLM, DataCollatorForLanguageModeling, EarlyStoppingCallback, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import evaluate
import torch

In [23]:
name_model = 'distilbert-base-uncased'

tokenizer = DistilBertTokenizer.from_pretrained(name_model)

In [24]:
collator = DataCollatorForLanguageModeling(tokenizer, mlm_probability=0.1)

In [71]:
sample_encoded = {
    'input_ids': [101, 2335, 15464, 2361, 3478, 2012, 2377, 28093, 13212, 2335, 15464, 2361, 4708, 13558, 3231, 2553, 28093, 13212, 3114, 8534, 4708, 4130, 2188, 7309, 2573, 15327, 2019, 19307, 3645, 2193, 2364, 11498, 7484, 1058, 22984, 7159, 2193, 1041, 8873, 2019, 19307, 5443, 27921, 2063, 2175, 2015, 27354, 2691, 13558, 3231, 2553, 1061, 19968, 2193, 10611, 2334, 15006, 2102, 3478, 3231, 2553, 28093, 13212, 2003, 8534, 2138, 1058, 2213, 8059, 5906, 5361, 6270, 2770, 6270, 0, 102],
    'special_tokens_mask': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
}

sample_encoded2 = {
    'input_ids': [101, 2335, 15464, 2361,  102, 0],
    'special_tokens_mask': [1, 0, 0, 0, 1, 1],
    'attention_mask': [1, 1, 1, 1, 1, 0]
}


sample = '[MASK] [MASK] at play deploy vm efi para virtual vmxnet number timestamp task wait for message auto install is completed appear in vm log serial timestamp log task path home worker workspace ansible cycle photon number x update ansible vsphere gos validation common vm wait log message yml number fatal localhost failed attempts number censored the output has been hidden due to the fact that no log true was specified for this result changed false timestamp task testing exit due to failure task path home worker workspace ansible cycle photon number x update ansible vsphere gos validation common test rescue yml number fatal localhost failed exit testing when exit testing when fail is set to true in test case deploy vm efi para virtual vmxnet number'
encoded = [2335, 15464, 2361, 3478, 2012, 2377, 28093, 13212, 2335, 15464, 2361, 4708, 13558, 3231, 2553, 28093, 13212, 3114, 8534, 4708, 4130, 2188, 7309, 2573, 15327, 2019, 19307, 3645, 2193, 2364, 11498, 7484, 1058, 22984, 7159, 2193, 1041, 8873, 2019, 19307, 5443, 27921, 2063, 2175, 2015, 27354, 2691, 13558, 3231, 2553, 1061, 19968, 2193, 10611, 2334, 15006, 2102, 3478, 3231, 2553, 28093, 13212, 2003, 8534, 2138, 1058, 2213, 8059, 5906, 5361, 6270, 2770, 6270]

In [26]:
collator([{
    'input_ids': [101, 2335, 15464, 2361, 3478, 2012, 2377, 28093, 13212, 2335, 15464, 2361, 4708, 13558, 3231, 2553, 28093, 13212, 3114, 8534, 4708, 4130, 2188, 7309, 2573, 15327, 2019, 19307, 3645, 2193, 2364, 11498, 7484, 1058, 22984, 7159, 2193, 1041, 8873, 2019, 19307, 5443, 27921, 2063, 2175, 2015, 27354, 2691, 13558, 3231, 2553, 1061, 19968, 2193, 10611, 2334, 15006, 2102, 3478, 3231, 2553, 28093, 13212, 2003, 8534, 2138, 1058, 2213, 8059, 5906, 5361, 6270, 2770, 6270, 102],
    'special_tokens_mask': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
}])

In [29]:
collator.__call__.__code__

<code object __call__ at 0x7f480f6e43a0, file "/opt/conda/lib/python3.8/site-packages/transformers/data/data_collator.py", line 35>

In [56]:
tokenizer.encode_plus(sample, padding="max_length", max_length=200, truncation=True, return_special_tokens_mask=True)
# tokenizer.pad([sample_encoded, sample_encoded2])
# tokenizer.pad([sample_encoded2])
# tokenizer.pad
# tokenizer.decode([2335, 15464, 2361, 3478, 2012, 2377])

In [72]:
def transfer_one_row_to_sample(tokens_raw, seq_max):
    tokens_0 = [101] + tokens_raw + [102]
    indexs_special_0 = [1] + [0 for _ in range(len(tokens_0) - 2)]+ [1]

    num_padding = seq_max - len(tokens_0)

    if num_padding:
        tokens_padded = tokens_0 + [0] * num_padding
        indexs_special_padded = indexs_special_0 + [1] * num_padding
    else:
        tokens_padded = tokens_0
        indexs_special_padded = indexs_special_0
    # end

    sample_0 = {'input_ids': tokens_padded, 'special_tokens_mask': indexs_special_padded}    # return_special_tokens_mask=True
    return sample_0
# end

In [75]:
len(transfer_one_row_to_sample(encoded, 258)['special_tokens_mask'])

258